In [1]:
import json
import logging
import re
import warnings
from datetime import datetime
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Generator, Literal, Type, TypeVar

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)


def create_path(path: str | Path) -> None:
    """
    Create parent directories for the given path if they don't exist.

    Parameters
    ----------
    path : str | Path
        The file path for which to create parent directories.

    """
    Path(path).parent.mkdir(parents=True, exist_ok=True)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/MyProjects/batch-process


In [4]:
from sqlalchemy import delete, insert, select, update

from schemas import EmailSchema
from schemas.db_models import EmailLog, get_db_session, init_db

Connected to 'test' environment database.


In [5]:
init_db()

In [ ]:
from datetime import datetime

from pydantic import BaseModel, Field, field_validator


class MyG(BaseModel):
    status: Literal["dispatched", "failed", "processing", "sent"] = Field(
        default="processing", description="Email status"
    )
    created_at: datetime = Field(
        default_factory=datetime.now, description="Creation time"
    )
    sent_at: datetime | None = Field(
        default_factory=datetime.now, description="Time sent"
    )
    number: float = Field(description="A number")

    @field_validator("sent_at", mode="before")
    def serialize_datetimes(cls, v: datetime) -> str:
        print(v)
        return v.isoformat()

    @field_validator("number", mode="before")
    def round_number(cls, v: float) -> float:
        print(v)
        return round(v, 2)

In [ ]:
example = MyG(status="sent", created_at=datetime.now(), number=2.33333)
print(example.model_dump())

2.33333
{'status': 'sent', 'created_at': datetime.datetime(2025, 7, 14, 19, 25, 52, 589239), 'sent_at': datetime.datetime(2025, 7, 14, 19, 25, 52, 589257), 'number': 2.33}


datetime.datetime

## [Docs](https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html)

### [Insert](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-bulk-insert-statements)

- Old API

```python
with get_db_session() as session:
    data_dict = input_data.to_data_model_dict()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}
```

<br>

- New API

```py
with get_db_session() as session:
    data_dict = input_data.to_data_model_dict()
    session.execute(insert(EmailLog), [data_dict])
```

In [6]:
input_data: EmailSchema = EmailSchema(
    recipient="marketing@client.com",
    subject="Partnership Proposal",
    body="We would like to discuss a potential partnership opportunity.",
)
console.print(input_data)

EmailSchema(
    recipient='marketing@client.com',
    subject='Partnership Proposal',
    body='We would like to discuss a potential partnership opportunity.',
    status='processing',
    created_at=datetime.datetime(2025, 7, 14, 19, 3, 24, 319222),
    sent_at=datetime.datetime(2025, 7, 14, 19, 3, 24, 319230)
)

In [7]:
with get_db_session() as session:
    data_dict = input_data.to_data_model_dict()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

StatementError: (builtins.TypeError) SQLite DateTime type only accepts Python datetime and date objects as input.
[SQL: INSERT INTO email_logs (recipient, subject, body, status, "createdAt", "sentAt") VALUES (?, ?, ?, ?, ?, ?)]
[parameters: [{'createdAt': '2025-07-14T19:03:24.319222', 'body': 'We would like to discuss a potential partnership opportunity.', 'sentAt': '2025-07-14T19:03:24.319230', 'status': 'processing', 'subject': 'Partnership Proposal', 'recipient': 'marketing@client.com'}]]

In [ ]:
input_data_2: EmailSchema = EmailSchema(
    recipient="emeka2@example.com",
    subject="test!!!",
    body="this is an example body",
    status="processing",
)
input_data_3: EmailSchema = EmailSchema(
    recipient="john.doe@example.com",
    subject="Meeting Reminder",
    body="Hi John, just a reminder about our meeting tomorrow at 10 AM.",
    status="processing",
)
input_data_4: EmailSchema = EmailSchema(
    recipient="info@company.org",
    subject="New Product Launch",
    body="Dear valued customer, check out our exciting new product!",
    status="sent",
    created_at=datetime(2025, 7, 10, 9, 0, 0),
    sent_at="2025-07-10T09:05:00",
)
console.print((input_data_2, input_data_3, input_data_4))

### [Bulk Insert](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-bulk-insert-statements)

- Old API

```py
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [_data.to_data_model_dict() for _data in (input_data_2, input_data_3, input_data_4)]
    session.bulk_insert_mappings(EmailLog, data_list)
```

<br>

- New API

```py
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [
        _data.to_data_model_dict()
        for _data in (input_data_2, input_data_3, input_data_4)
    ]
    session.execute(insert(EmailLog), data_list)
```

In [ ]:
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [
        _data.to_data_model_dict() for _data in (input_data_2, input_data_3, input_data_4)
    ]
    session.execute(insert(EmailLog), data_list)

### Select

In [ ]:
# Select a single record
with get_db_session() as session:
    statement = select(EmailLog).where(EmailLog.id == 1, EmailLog.status == "pending")
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

In [ ]:
# Select all records
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

### [Update](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [ ]:
with get_db_session() as session:
    statement = (
        update(EmailLog)
        .where(EmailLog.id == 1)
        .values(status="sent", sent_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    )
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

### [Delete](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [ ]:
with get_db_session() as session:
    statement = delete(EmailLog).where(EmailLog.id == 2)
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

In [ ]:
from config import app_config

In [ ]:
beat_dict: dict[str, dict[str, Any]] = dict(
    app_config.celery_config.beat_config.beat_schedule.model_dump().items()
)

# Add the health_check
beat_dict["health_check"] = (
    app_config.celery_config.beat_config.health_check.model_dump()
)


console.print(beat_dict)

In [ ]:
app_config.celery_config.beat_config.beat_schedule.model_dump().items()